# Processing and classifying images

In [ ]:
from transformers import image_transforms
from transformers import pipeline
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from datasets import Audio
from datasets import Dataset, load_dataset
import librosa
from evaluate import load
import pandas as pd

## Processing image data

In [ ]:
original_image = Image.open("images/fashion.jpeg")

# Create the numpy array
image_array = np.array(original_image)

imgplot = plt.imshow(image_array)
plt.show()

In [ ]:
# Crop the center of the image
cropped_image = image_transforms.center_crop(image=image_array, size=(200, 200))

imgplot = plt.imshow(cropped_image)
plt.show()

## Creating an image classifier

In [ ]:
cropped_pil_image = Image.fromarray(cropped_image)
print(cropped_pil_image)

In [ ]:
# Create the pipeline
image_classifier = pipeline(task="image-classification", 
            model="abhishek/autotrain_fashion_mnist_vit_base")

# Predict the class of the image
results = image_classifier(cropped_pil_image)

# Print the results
print(results[0]["label"])

## What about the original image?

In [ ]:
# Predict the class of the image
results = image_classifier(original_image)

# Print the results
print(results[0]["label"])

# Question answering and multi-modal tasks

## Document question and answering

In [ ]:
# Create the pipeline
dqa = pipeline(task="document-question-answering", model="naver-clova-ix/donut-base-finetuned-docvqa")

# Set the image and question
image = "images/document.jpg"
question = "Which meeting is this document about?"

# Get the answer
results = dqa(image=image, question=question)

print(results)

## Visual question and answering

In [ ]:
image = "images/fashion.jpeg"
question = "What is the model wearing in this image?"

# Create pipeline
vqa = pipeline(task="visual-question-answering", model="dandelin/vilt-b32-finetuned-vqa")

# Use image and question in vqa
results = vqa(image=image, question=question)

print(results)

# Audio classification

## Resampling audio files

In [ ]:
dataset = load_dataset("common_language")
dataset = dataset["train"]
dataset = dataset.select([0, 1, 2, 3, 4, 5, 6])

# Save the old sampling rate
old_sampling_rate = dataset[1]['audio']['sampling_rate']

# Resample the audio files
audio_file = dataset.cast_column("path", Audio(sampling_rate=16_000))

# Compare the old and new sampling rates
print("Old sampling rate:", old_sampling_rate)
print("New sampling rate:", dataset[1]['audio']['sampling_rate'])

## Filtering out audio files

In [ ]:
dataset = Dataset.from_dict({"path": ["audios/audio1.mp3", "audios/audio2.mp3", "audios/audio3.mp3"]}).cast_column("path", Audio())

# Create a list of durations
old_durations_list = []

# Loop over dataset
for row in dataset["path"]:
    old_durations_list.append(librosa.get_duration(path=row["path"]))

# Creat a new column
dataset = dataset.add_column("duration", old_durations_list)

# Filter the dataset
filtered_dataset = dataset.filter(lambda d: d < 60.0, input_columns=["duration"], keep_in_memory=True)

# Save new durations
new_durations_list = filtered_dataset["duration"]

print("Old duration:", np.mean(old_durations_list))
print("New duration:", np.mean(new_durations_list))

## Classifying audio files

In [ ]:
dataset = load_dataset("common_language")
dataset = dataset["train"]
dataset = dataset.select([0, 1, 2, 3, 4, 5, 6])

# Create the pipeline
classifier = pipeline(task="audio-classification", model="facebook/mms-lid-126")

# Extract the sample
audio = dataset[0]['audio']['array']
sentence = dataset[0]["sentence"]

# Predict the language
prediction = classifier(audio)

print(f"Predicted language is '{prediction[0]['label'].upper()}' for the sentence '{sentence}'")

# Automatic speech recognition

## Instantiating an ASR pipeline

In [ ]:
example = dataset[6]

# Create an ASR pipeline using Meta's wav2vec model
meta_asr = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

# Predict the text from the example audio
meta_pred = meta_asr(example["audio"]["array"])["text"].lower()

# Repeat for OpenAI's Whisper model
open_asr = pipeline("automatic-speech-recognition", model="openai/whisper-tiny")
open_pred = open_asr(example["audio"]["array"])["text"].lower()

# Print the prediction from both models
print("META:", meta_pred)
print("OPENAI:", open_pred)

## Word error rate

In [ ]:
# Create the word error rate metric
wer = load("wer")

# Save the true sentence of the example
true_sentence = example["sentence"].lower()

# Compute the wer for each model prediction
meta_wer = wer.compute(predictions=[meta_pred], references=[true_sentence])
open_wer = wer.compute(predictions=[open_pred], references=[true_sentence])

print(f"The WER for the Meta model is {meta_wer} and for the OpenAI model is {open_wer}")

## Iterating over a dataset

In [ ]:
# Create the data function
def data(n=3):
    for i in range(n):
        yield dataset[i]["audio"]["array"], dataset[i]["sentence"].lower()
        
# Predict the text for the audio file with both models
output = []
for audio, sentence in data():
    meta_pred = meta_asr(audio)["text"].lower()
    open_pred = open_asr(audio)["text"].lower()
    # Append to output list
    output.append({"sentence": sentence, "metaPred": meta_pred, "openPred": open_pred})

output_df = pd.DataFrame(output)

# Compute the WER for both models
metaWER = wer.compute(predictions=output_df["metaPred"], references=output_df["sentence"])
openWER = wer.compute(predictions=output_df["openPred"], references=output_df["sentence"])

# Print the WER
print(f"The WER for the meta model is {metaWER} and for the open model is {openWER}")